In [67]:
%%capture
!pip install plotly
!pip install alpha_vantage
!pip install quandl
!pip install --upgrade plotly
!pip install --upgrade notebook
!pip install arch

In [80]:
from alpha_vantage.timeseries import TimeSeries
import quandl
import matplotlib.pyplot as plt
import config 
import plotly.express as px
import pandas as pd
import os
import numpy as np
import datetime
import plotly.io as pio
pio.renderers.default = 'notebook'
from arch import arch_model


In [3]:
def write_date_to_file(file_path):
    today_date = datetime.datetime.today().strftime("%Y-%m-%d")
    with open(file_path, "w") as file:
        file.write(today_date)

# Function to read the date from the file
def read_date_from_file(file_path):
    with open(file_path, "r") as file:
        saved_date = file.read().strip()
    return saved_date



In [118]:

# Initialize the TimeSeries object with your API key
ts = TimeSeries(key=config.api_key, output_format='pandas')

subfolder='data/'

# Define the symbols and their corresponding dollar investments in your portfolio




def plot_portfolio_growth(portfolio,investment_date):

    date_path = 'last_fetched.txt'

    #### Check if the data was recently updated.

    if not os.path.exists(date_path):
        write_date_to_file(date_path)
    saved_date = read_date_from_file(date_path)
    today_date = datetime.datetime.today().strftime("%Y-%m-%d")
    # Compare the saved date and today's date
    if saved_date < today_date:
        rewrite_data = True
        write_date_to_file(date_path)
    else:
        rewrite_data = False
        


    initial_investment = sum(portfolio.values())

    # Create an empty DataFrame to store the daily returns
    global portfolio_returns
    portfolio_returns = pd.DataFrame()

    # Initialize a cumulative portfolio return variable
    cumulative_portfolio_return = 0

    # Fetch daily historical stock data and calculate returns for each symbol
    for symbol, investment in portfolio.items():    
        data_path = os.path.join(subfolder, 'portfolio_data_' + symbol.replace('.','_') + '.parquet')
        print("rewriting data? : ",((not os.path.exists(data_path)) | rewrite_data))

    # To prevent repeated API calls
        if ((not os.path.exists(data_path)) | rewrite_data):
            data, _ = ts.get_weekly_adjusted(symbol=symbol)
#            data,_ = ts.get_intraday(symbol=symbol, interval='5min', outputsize='full')
            data.to_parquet(data_path)
        else:

            data = pd.read_parquet(data_path)


        data = data.sort_values(by='date')




        data['Returns'] = data['5. adjusted close'].astype(float).pct_change()
        data = data[data.index >= investment_date]  # Filter data from the start date
        data=data.reset_index()
        data.loc[0,'investment_value'] = investment
        data['holding_size'] = investment/data.loc[0,'4. close']
        data['investment_value'] = data['holding_size'] * data['4. close']

        portfolio_returns[symbol] = data['investment_value']
        
    portfolio_returns = portfolio_returns.dropna()
    
    global portfolio_returns_df
    portfolio_returns_df = pd.concat([portfolio_returns,data['date']],axis=1)




    # Calculate the cumulative portfolio returns
    global cumulative_portfolio_returns
    cumulative_portfolio_returns = portfolio_returns.sum(axis=1)

    # Calculate the total portfolio value in dollar terms
    portfolio_value = pd.concat([cumulative_portfolio_returns,data['date']],axis=1)
    portfolio_value.columns=['value','date']


    # Set a dark theme style
    plt.style.use('dark_background')

    # Create a figure and axis
    plt.figure(figsize=(12, 6))
    ax = plt.gca()

    # Plot the data with a green line and circular markers
    plt.plot(portfolio_value['date'], portfolio_value['value'], label='Portfolio Value ($)', color='red', marker='o', markersize=3)

    # Add a title with white text
    plt.title('Growth in Portfolio Value Over Time', fontsize=16, color='white')

    # Customize the x-axis and y-axis labels with white text
    plt.xlabel('Date', fontsize=12, color='white')
    plt.ylabel('Portfolio Value', fontsize=12, color='white')

    # Add a grid with dashed lines and adjust transparency
    plt.grid(True, linestyle='--', alpha=0.4)

    # Customize the legend with white text
    plt.legend(loc='upper left', fontsize=12, edgecolor='white', facecolor='black')

    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, color='white')

    # Set the background color to black
    ax.set_facecolor('black')

    # Display the plot
    plt.tight_layout()
    plt.show()
    


    df = portfolio_returns_df

    # Convert the 'date' column to datetime
    df['date'] = pd.to_datetime(df['date'])

    # Set the 'date' column as the index (optional)
    # df.set_index('date', inplace=True)

    # Set a dark theme style
    plt.style.use('dark_background')

    # Create a figure and axis
    plt.figure(figsize=(12, 6))
    ax = plt.gca()

    # Plot the three time series on the same plot with custom colors

    stocks = list(portfolio_1.keys())
    colors = plt.cm.viridis(np.linspace(0, 1, len(stocks)))

    for i,col in enumerate(stocks):
        color = colors[i]
        plt.plot(df['date'], df[col], label=col, color=color, linestyle='-', marker='o', markersize=3)


    # Customize the plot with white text and a dark background
    plt.title('Individual Portfolio holdings', fontsize=16, color='white')
    plt.xlabel('Date', fontsize=12, color='white')
    plt.ylabel('Value', fontsize=12, color='white')
    plt.legend(fontsize=12, edgecolor='white', facecolor='black')
    plt.grid(True, linestyle='--', alpha=0.4)
    plt.xticks(rotation=45, color='white')
    ax.set_facecolor('black')

    # Show the plot
    plt.tight_layout()
    plt.show()
    
    return(portfolio_value)

In [119]:


### Try with different portfolio combinations
portfolio_1 = {
    'VEQT.TO': 0, 
    'AAPL': 1000,  
    'TSLA': 4000
}

portfolio_2 = {
    'VEQT.TO': 4000, 
    'AAPL': 1000,  
    'TSLA': 1000
}

investment_date = '2019-02-01'


for i in [portfolio_1,portfolio_2]:
    plot_portfolio_growth(i,investment_date)

rewriting data? :  True


ValueError: Thank you for using Alpha Vantage! You have reached the 100 requests/day limit for your free API key. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.

In [115]:
x,_ = ts.get_weekly_adjusted(symbol='TSLA')

x=x.reset_index()
x[x['date']>'2019-01-01']

,date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount
0,2023-10-03,244.81,254.2799,242.620,246.53,246.5300,225795707.0,0.0
1,2023-09-29,243.38,254.7700,234.580,250.22,250.2200,588808971.0,0.0
2,2023-09-22,271.16,273.9300,244.480,244.88,244.8800,575237587.0,0.0
3,2023-09-15,264.27,278.9800,260.610,274.39,274.3900,663743610.0,0.0
4,2023-09-08,245.00,258.0000,243.265,248.50,248.5000,480301845.0,0.0
...,...,...,...,...,...,...,...,...
244,2019-02-01,292.91,316.1000,287.750,312.21,20.8140,42147975.0,0.0
245,2019-01-25,304.82,308.0000,279.280,297.04,19.8027,39866967.0,0.0
246,2019-01-18,342.38,352.0000,299.730,302.26,20.1507,43823109.0,0.0
247,2019-01-11,321.72,348.4100,317.750,347.26,23.1507,31088092.0,0.0


In [106]:
portfolio_returns

,VEQT.TO,AAPL,TSLA
0,4000.000000,1000.000000,1000.000000
1,4036.386449,1006.336663,1039.270787
2,4043.914680,998.509020,1017.718293
3,4081.555834,1024.228418,1012.121789
4,4090.338770,1019.531832,1014.789059
...,...,...,...
666,4301.129235,1312.658417,294.102237
667,4328.732748,1281.944237,290.684798
668,4337.515684,1270.538244,286.374299
669,4318.695107,1272.476517,293.375883


In [78]:
portfolio_1.items

<function dict.items>

In [101]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=IBM&interval=5min&apikey=demo'
r = requests.get(url)
data = r.json()

pd.DataFrame(data).head()

,Meta Data,Time Series (5min)
1. Information,"Intraday (5min) open, high, low, close prices ...",NaN
2. Symbol,IBM,NaN
3. Last Refreshed,2023-10-03 19:55:00,NaN
4. Interval,5min,NaN
5. Output Size,Compact,NaN
